In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.impute import KNNImputer
import pickle

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Handling Errant Values 

In [2]:
raw_file = "/home/ec2-user/SageMaker/Team-5/Submission/Datasets/TN/TN_allinone01122.csv"
df = pd.read_csv(raw_file)

/tmp/ipykernel_28550/1749200401.py:2: DtypeWarning: Columns (2,3,4,5,9,14,16,20,21,23,24,26,27,29,30,32,34,35,37,39,40,43,44,47,48,51,52,55,56,59,60,61,62,63,64,65,66,67,68,69,70,71,72,74,77,80,81,82,84,85,93,106,107,108,109,110,111,112,113,114,116,118,121,125,127,128,129,131,132,136,137,141,142,146,147,151,152,156,157,161,162,166,167,168,169,170,171,172,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,252,255,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,299,301,302,303,304,306,309,312,316,318,321,323,324,327,329,331,333,334,335,337,342,347,348,350,351,352,353,354,355,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 706 entries, MaskID to INST
dtypes: float64(350), int64(1), object(355)
memory usage: 1.4+ GB


In [4]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 706 entries, MaskID to INST
dtypes: float64(350), int64(1), object(355)
memory usage: 1.4+ GB


In [7]:
## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

In [8]:
## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')



Hemoglobin_process = ['HemoglobinResults']
df[Hemoglobin_process] = df[Hemoglobin_process].apply(lambda x: x.where(x <= 30, x / 10))

Hematocrit_process = ['HematocritResults']
df[Hematocrit_process] = df[Hematocrit_process].apply(lambda x: x.where(x >= 1, x * 100))


column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

df.rename(columns={'LowestGlucose_unitl': 'LowestGlucose_unit'}, inplace=True)
print(df.shape)


(259168, 706)


In [9]:
existing_column_index = df.columns.get_loc('HemoglobinResults')
df.insert(existing_column_index + 1, 'HemoglobinResults_Units', "NaN")
df.loc[df['HemoglobinResults'].notna(), 'HemoglobinResults_Units'] = "mg/dL"

existing_column_index = df.columns.get_loc('HbA1c')
df.insert(existing_column_index + 1, 'HbA1c_Units', "NaN")
df.loc[df['HbA1c'].notna(), 'HbA1c_Units'] = "Hb%"

existing_column_index = df.columns.get_loc('MCHresults')
df.insert(existing_column_index + 1, 'MCHresults_Units', "NaN")
df.loc[df['MCHresults'].notna(), 'MCHresults_Units'] = "pg"

existing_column_index = df.columns.get_loc('MCHCresults')
df.insert(existing_column_index + 1, 'MCHCresults_Units', "NaN")
df.loc[df['MCHCresults'].notna(), 'MCHCresults_Units'] = "g/dL"

existing_column_index = df.columns.get_loc('EosinophilsResults')
df.insert(existing_column_index + 1, 'EosinophilsResults_Units', "NaN")
df.loc[df['EosinophilsResults'].notna(), 'EosinophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResults')
df.insert(existing_column_index + 1, 'BasophilsResults_Units', "NaN")
df.loc[df['BasophilsResults'].notna(), 'BasophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResults')
df.insert(existing_column_index + 1, 'LymphocytesResults_Units', "NaN")
df.loc[df['LymphocytesResults'].notna(), 'LymphocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('HematocritResults')
df.insert(existing_column_index + 1, 'HematocritResults_Units', "NaN")
df.loc[df['HematocritResults'].notna(), 'HematocritResults_Units'] = "% of blood"

existing_column_index = df.columns.get_loc('MCVresults')
df.insert(existing_column_index + 1, 'MCVresults_Units', "NaN")
df.loc[df['MCVresults'].notna(), 'MCVresults_Units'] = "fL"

existing_column_index = df.columns.get_loc('MonocytesResults')
df.insert(existing_column_index + 1, 'MonocytesResults_Units', "NaN")
df.loc[df['MonocytesResults'].notna(), 'MonocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResults')
df.insert(existing_column_index + 1, 'NeutrophilsResults_Units', "NaN")
df.loc[df['NeutrophilsResults'].notna(), 'NeutrophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('WBCresults')
df.insert(existing_column_index + 1, 'WBCresults_Units', "NaN")
df.loc[df['WBCresults'].notna(), 'WBCresults_Units'] = "x10^9 cells/L"

existing_column_index = df.columns.get_loc('Rbcresults')
df.insert(existing_column_index + 1, 'Rbcresults_Units', "NaN")
df.loc[df['Rbcresults'].notna(), 'Rbcresults_Units'] = "x10^12 cells/L"

existing_column_index = df.columns.get_loc('EosinophilsResultsOR')
df.insert(existing_column_index + 1, 'EosinophilsResultsOR_Units', "NaN")
df.loc[df['EosinophilsResultsOR'].notna(), 'EosinophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResultsOR')
df.insert(existing_column_index + 1, 'LymphocytesResultsOR_Units', "NaN")
df.loc[df['LymphocytesResultsOR'].notna(), 'LymphocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResultsOR')
df.insert(existing_column_index + 1, 'BasophilsResultsOR_Units', "NaN")
df.loc[df['BasophilsResultsOR'].notna(), 'BasophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('MonocytesResultsOR')
df.insert(existing_column_index + 1, 'MonocytesResultsOR_Units', "NaN")
df.loc[df['MonocytesResultsOR'].notna(), 'MonocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResultsOR')
df.insert(existing_column_index + 1, 'NeutrophilsResultsOR_Units', "NaN")
df.loc[df['NeutrophilsResultsOR'].notna(), 'NeutrophilsResultsOR_Units'] = "% of WBC"

In [10]:
columns_to_filter = ['Rbcresults', 'HemoglobinResults',
                  'HematocritResults', 'MCVresults',
                  'PlateletCountResultsNormal', 'MCHresults',
                  'MCHCresults', 'WBCresults',
                  'NeutrophilsResultsOR', 'LymphocytesResultsOR',
                  'MonocytesResultsOR', 'EosinophilsResultsOR',
                  'BasophilsResultsOR']
filter_columns = df.columns[df.columns.str.contains('ResultsNormal', case=False)].tolist()
element_to_remove = 'PlateletCountResultsNormal'
filter_columns.remove(element_to_remove)

print(columns_to_filter)
print(filter_columns)

for col, filter_col in zip(columns_to_filter, filter_columns):
    filter_mask = df[filter_col] == 'Yes'
    lower_bound = df[col].quantile(0.0025)
    upper_bound = df[col].quantile(0.9975)
     # Apply np.where only to the relevant subset
    df.loc[~filter_mask, col] = np.where((df.loc[~filter_mask, col] < lower_bound) | 
                                         (df.loc[~filter_mask, col] > upper_bound), 
                                         np.nan, 
                                         df.loc[~filter_mask, col])
print(df)

['Rbcresults', 'HemoglobinResults', 'HematocritResults', 'MCVresults', 'PlateletCountResultsNormal', 'MCHresults', 'MCHCresults', 'WBCresults', 'NeutrophilsResultsOR', 'LymphocytesResultsOR', 'MonocytesResultsOR', 'EosinophilsResultsOR', 'BasophilsResultsOR']
['RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal']
        MaskID    Visit_Dt _15Aug2011ConsentForResidualSa  \
0       200000  06/29/2007                            NaN   
1       200003  06/29/2015                            Yes   
2       200004  08/11/2015                            Yes   
3       200007  11/21/2016                            Yes   
4       200010  10/21/2016                            Yes   
...        ...         ...                         

In [11]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 724 entries, MaskID to Weight_kg_units
dtypes: float64(348), int64(1), object(375)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,Spring2019ConsentDateMonth,Spring2019ConsentDateDay,Spring2019ConsentDateYear,DateOfBloodDrawDay,DateOfBloodDrawMonth,...,ALP,ALB,UREA,AST,CPEP,INST,Height_cm,Height_cm_units,Weight_kg,Weight_kg_units
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg


In [12]:
# Replace both "." and " ." with NaN
df = df.replace({'.': np.nan, ' .': np.nan, '  .':np.nan})

# Date column harmonization

# 1. Columns of Date

#### Steps of date column harmonization:
1. Filter out date column sets based on their headers
2. Create one date column to replace one set of date columns.
3. assign values in the format of date_time to the new created column.

In [13]:
merged_df=df
#This is a tool to find columns based on names of their headers and check their values. 
contain_list_0 = ['SerumKetonesDay'] 
# Example strings to exclude
not_contain_list_0 = []     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

Found  0 columns:  []
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame
None


## 1.1 Day, Month, Year

In [14]:
# Filter all columns with "Day", "Month", "Year" as date sets
# in lower case for case-insensitive comparison
# Example strings to include
contain_list_1 = ['Day', "Month", 'Year'] 
# Example strings to exclude
not_contain_list_1 = ['LastYear', 'Last6Months', '3Days', 'Birth', 'MostRecentHbA1c', 'PastYear', 
                      'ChangeInDrugStatusEffective', 'ParticipantDateDispensed', 'MechanisticSampleCollection',
                     'Polydipsia', 'Polyuria', '10days', 'ADHPsoriasisDateofDiagnosis', 'CeliacSprueGlutenDiagnosis',
                     'Datethebloodsamplewasdrawn', 'MHInflammatoryBowelDisease', 'ADHAlopeciaDateofDiagnosis', 
                      'ADHOtherDateofDiagnosis', 'ADHVitiligoDateofDiagnosis', 'VitiligoDiagnosis', 'NextYear',
                      'Polyphagia', 'Fatigue', 'monthly', 'Ketoacidosis', 'UnexplainedWeightLoss', 'HbA1crecordDate']     

# Converting each string in the lists to lower case
contain_list_1_low = [string.lower() for string in contain_list_1]
not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col.lower() for contain in contain_list_1_low) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list_1_low)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
#for col in filtered_columns:
#    print(f"Top 20 values for column {col}:")
#    print(df[col].value_counts().nlargest(20))
#    print()

Found  42 columns:  ['Spring2019ConsentDateMonth', 'Spring2019ConsentDateDay', 'Spring2019ConsentDateYear', 'DateOfBloodDrawDay', 'DateOfBloodDrawMonth', 'DateOfBloodDrawYear', 'DateReportedDay', 'DateReportedMonth', 'DateReportedYear', 'ScreeningDay', 'ScreeningMonth', 'ScreeningYear', 'ConsentSignedDay', 'ConsentSignedMonth', 'ConsentSignedYear', 'DateConsentSignedDay', 'DateConsentSignedMonth', 'DateConsentSignedYear', 'SamplesCollectionDay', 'SamplesCollectionMonth', 'SamplesCollectionYear', 'LastHbA1cDay', 'LastHbA1cMonth', 'LastHbA1cYear', 'WrittenInformedConsentDay', 'WrittenInformedConsentMonth', 'WrittenInformedConsentYear', 'Type1DiabetesDiagnosisMonth', 'Type1DiabetesDiagnosisDay', 'Type1DiabetesDiagnosisYear', 'FluVaccinationDateMonth', 'FluVaccinationDateDay', 'FluVaccinationDateYear', 'DateTestsRunMonth', 'DateTestsRunDay', 'DateTestsRunYear', 'StudyDrugReturnedDay', 'StudyDrugReturnedMonth', 'StudyDrugReturnedYear', 'StudyDrugDispensedDay', 'StudyDrugDispensedMonth', 'St

#### Check month and year columns based on day columns

In [15]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "Day" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("Day", "Month") for name in day_column_list_1]
year_column_list_1 = [name.replace("Day", "Year") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  14
No. of month columes in wrong naming format: 0
Month columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


#### Check day and year columns based on month columns

In [16]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "Month" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("Month", "Day") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("Month", "Year") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  14
No. of day columes in wrong naming format: 0
Day columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [17]:
import numpy as np
# Convert 'NA' strings to np.nan
def convert_na(value):
    if pd.isna(value) or value == 'NA':  # using pandas function to check for NA
        return np.nan   # converting to numpy NaN
    else:
        return value

# Function to convert month from string or number to numeric month, handling missing values
def month_to_num(month):
    if pd.isna(month):
        return np.nan
    try:
        # Handle string representation
        if isinstance(month, str):
            datetime_object = pd.to_datetime(month, format='%b')
            return datetime_object.month
        # Handle numeric representation
        elif isinstance(month, (int, float)):
            return int(month)
    except ValueError:
        return np.nan
    
# Create a date column and assign values by combine date columns with similar header
def create_date_column(df, year_col, original_cols):
    parts = year_col.split('Year')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Month' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        df[date_col] = pd.NaT  # Initialize column with NaT (Not-a-Time) for datetime objects
        problematic_rows = []

        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    continue  # NaNs are already handled by NaT initialization
                else:
                    year = int(row[year_col])
                    month = int(month_to_num(row[month_col]))
                    day = int(row[day_col])
                    df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = pd.NaT

        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [18]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df.loc[:, col] = df[col].apply(convert_na)

# Iterating over columns and creating date columns
for col in year_column_list_1:
    create_date_column(df, col, filtered_date_columns_1)

print(df.info())
df.head()

/tmp/ipykernel_28550/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-07-22' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_28550/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-11-08' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_28550/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-11-08' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 738 entries, MaskID to StudyDrugDispensedDate
dtypes: float64(353), int64(1), object(384)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,Spring2019ConsentDateMonth,Spring2019ConsentDateDay,Spring2019ConsentDateYear,DateOfBloodDrawDay,DateOfBloodDrawMonth,...,ConsentSignedDate,DateConsentSignedDate,SamplesCollectionDate,LastHbA1cDate,WrittenInformedConsentDate,Type1DiabetesDiagnosisDate,FluVaccinationDateDate,DateTestsRunDate,StudyDrugReturnedDate,StudyDrugDispensedDate
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2007-06-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 738 entries, MaskID to StudyDrugDispensedDate
dtypes: float64(353), int64(1), object(384)
memory usage: 1.4+ GB


In [20]:
columns_to_drop = [col for col in filtered_date_columns_1]

print(len(columns_to_drop))
df.drop(columns=columns_to_drop, inplace=True)
df.info()

42
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 696 entries, MaskID to StudyDrugDispensedDate
dtypes: float64(327), int64(1), object(368)
memory usage: 1.3+ GB


## 1.2 Month, Year

In [21]:
column_pairs = [
    ('BirthMonth', 'BirthYear'),
    ('DateOfBirthMonth', 'DateOfBirthYear'),
    ('HbA1crecordDateMonth', 'HbA1crecordDateYear')
]

In [22]:
# Modified convert_to_datetime function to handle missing columns
def convert_to_datetime(row, month_col, year_col):
    try:
        if pd.notna(row[month_col]) and pd.notna(row[year_col]):
            month = row[month_col].capitalize()  # Capitalize to match standard abbreviations
            year = int(row[year_col])
            return pd.to_datetime(f"{month} {year}", format='%b %Y').strftime('%b %Y')
        else:
            return np.nan
    except (ValueError, TypeError):
        return np.nan
    
def safe_convert_to_datetime(df, month_col, year_col):
    if month_col in df.columns and year_col in df.columns:
        return df.apply(lambda row: convert_to_datetime(row, month_col, year_col), axis=1)
    else:
        return pd.Series([np.nan] * len(df))

# Create a new DataFrame for the new columns
new_columns_df = pd.DataFrame()

In [23]:
# Iterate over each pair and create a new column
for month_col, year_col in column_pairs:
    new_col_name = month_col.replace('Month', '') + 'Date'
    new_columns_df[new_col_name] = safe_convert_to_datetime(df, month_col, year_col)

# Check for missing columns and print them
for month_col, year_col in column_pairs:
    if month_col not in df.columns or year_col not in df.columns:
        print(f"Column pair missing: {month_col}, {year_col}")

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, new_columns_df], axis=1)

# Check the result
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 699 entries, MaskID to HbA1crecordDateDate
dtypes: float64(327), int64(1), object(371)
memory usage: 1.3+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,TimeOfCollectionHH,TimeOfCollectionMM,DateResultsReportedByLabUnknow,TimeResultsProcessedByLabHH,TimeResultsProcessedByLabMM,...,LastHbA1cDate,WrittenInformedConsentDate,Type1DiabetesDiagnosisDate,FluVaccinationDateDate,DateTestsRunDate,StudyDrugReturnedDate,StudyDrugDispensedDate,BirthDate,DateOfBirthDate,HbA1crecordDateDate
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,May 1987,NaN,NaN
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Sep 1994,NaN,NaN
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Aug 1970,NaN,NaN
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Jan 2009,NaN,NaN
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Sep 1982,NaN,NaN


In [24]:
#df = df.drop(['MostRecentHbA1cDate'], axis=1)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 699 entries, MaskID to HbA1crecordDateDate
dtypes: float64(327), int64(1), object(371)
memory usage: 1.3+ GB


In [26]:
columns_to_drop = [col for pair in column_pairs for col in pair]

for i in columns_to_drop:
    if i in df.columns:
        df.drop(columns=i, inplace=True)
        
    else:
        continue
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 693 entries, MaskID to HbA1crecordDateDate
dtypes: float64(324), int64(1), object(368)
memory usage: 1.3+ GB


## 1.3 DD, MM, YYYY

In [27]:
# Filter all columns with "DD", "MM", or "YYYY"
contain_list_1 = ["DD", "MM", "YYYY"] 
# Example strings to exclude
not_contain_list_1 = ['Autoimmuned', 'MMTT', 'TTT', 'TimeOfCollection', 'TimeResultsProcessedByLab']     

# Converting each string in the lists to lower case
#contain_list_1_low = [string.lower() for string in contain_list_1]
#not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col for contain in contain_list_1) and 
                    all(not_contain not in col for not_contain in not_contain_list_1)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

Found  9 columns:  ['DateRecordedDD1', 'DateRecordedMM1', 'DateRecordedYYYY1', 'DateRecordedDD2', 'DateRecordedMM2', 'DateRecordedYYYY2', 'DateRecordedDD3', 'DateRecordedMM3', 'DateRecordedYYYY3']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DateRecordedDD1    855 non-null    float64
 1   DateRecordedMM1    855 non-null    object 
 2   DateRecordedYYYY1  854 non-null    float64
 3   DateRecordedDD2    856 non-null    float64
 4   DateRecordedMM2    856 non-null    object 
 5   DateRecordedYYYY2  855 non-null    float64
 6   DateRecordedDD3    856 non-null    float64
 7   DateRecordedMM3    856 non-null    object 
 8   DateRecordedYYYY3  855 non-null    float64
dtypes: float64(6), object(3)
memory usage: 17.8+ MB
None


In [28]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "DD" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("DD", "MM") for name in day_column_list_1]
year_column_list_1 = [name.replace("DD", "YYYY") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  3
No. of month columes in wrong naming format: 0
Month columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [29]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "MM" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("MM", "DD") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("MM", "YYYY") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  3
No. of day columes in wrong naming format: 0
Day columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [30]:
import pandas as pd
import numpy as np

# Function to convert month from string to number, handling missing values
def month_to_num_1(month_str):
    if pd.isna(month_str):
        return np.nan
    try:
        datetime_object = pd.to_datetime(month_str, format='%b')
        return datetime_object.month
    except ValueError:
        return np.nan
    
def create_date_column_2(df, year_col, original_cols):
    parts = year_col.split('YYYY')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'DD' + suffix
    month_col = base_col_name + 'MM' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [31]:
def convert_na(value):
    if pd.isna(value) or value == 'NA':  # using pandas function to check for NA
        return np.nan   # converting to numpy NaN
    else:
        return value

In [32]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in filtered_date_columns_1:
    if 'YYYY' in col:
        create_date_column_2(df, col, original_cols)

print(df.info())
df.head()

/tmp/ipykernel_28550/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-02-07' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_28550/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-02-08' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_28550/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-02-09' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 696 entries, MaskID to DateRecordedDate3
dtypes: float64(324), int64(1), object(371)
memory usage: 1.3+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,TimeOfCollectionHH,TimeOfCollectionMM,DateResultsReportedByLabUnknow,TimeResultsProcessedByLabHH,TimeResultsProcessedByLabMM,...,FluVaccinationDateDate,DateTestsRunDate,StudyDrugReturnedDate,StudyDrugDispensedDate,BirthDate,DateOfBirthDate,HbA1crecordDateDate,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,May 1987,NaN,NaN,NaN,NaN,NaN
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,Sep 1994,NaN,NaN,NaN,NaN,NaN
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,Aug 1970,NaN,NaN,NaN,NaN,NaN
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,Jan 2009,NaN,NaN,NaN,NaN,NaN
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,Sep 1982,NaN,NaN,NaN,NaN,NaN


In [33]:
columns_to_drop = [col for col in filtered_date_columns_1]

len(columns_to_drop)

for i in columns_to_drop:
    if i in df.columns:
        df.drop(columns=i, inplace=True)
    else:
        continue
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 687 entries, MaskID to DateRecordedDate3
dtypes: float64(318), int64(1), object(368)
memory usage: 1.3+ GB


## 1.4 day, mon, yea

In [34]:
contain_list_0 = ['mon'] 
# Example strings to exclude
not_contain_list_0 = ['month', 'mont', 'Monitor', 'Mono', 'PastSixMon', 'Whendidthepneumoniaoccur', 'ReasonsNotInterested', 'Monetary']     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

Found  4 columns:  ['MechanisticSampleCollectionMon', 'ParticipantDateDispensedMon1_1', 'ParticipantDateDispensedMon2_1', 'ParticipantDateDispensedMon3_1']
<class 'pandas.core.frame.DataFrame'>
Index: 3923 entries, 190 to 257195
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   MechanisticSampleCollectionMon  3664 non-null   object
 1   ParticipantDateDispensedMon1_1  259 non-null    object
 2   ParticipantDateDispensedMon2_1  195 non-null    object
 3   ParticipantDateDispensedMon3_1  110 non-null    object
dtypes: object(4)
memory usage: 153.2+ KB
None
Top 20 values for column MechanisticSampleCollectionMon:
MechanisticSampleCollectionMon
Apr    376
Jun    335
May    334
Mar    332
Dec    331
Jul    317
Feb    308
Aug    292
Jan    290
Oct    278
Nov    277
Sep    194
Name: count, dtype: int64

Top 20 values for column ParticipantDateDispensedMon1_1:
ParticipantDateDispensedMon1

In [35]:
col_list = ['ParticipantDateDispensedDay1_1', 'ParticipantDateDispensedMon1_1', 'ParticipantDateDispensedYea1_1',
'ParticipantDateDispensedDay2_1', 'ParticipantDateDispensedMon2_1', 'ParticipantDateDispensedYea2_1',
'ParticipantDateDispensedDay3_1', 'ParticipantDateDispensedMon3_1', 'ParticipantDateDispensedYea3_1',
'MechanisticSampleCollectionDay', 'MechanisticSampleCollectionMon', 'MechanisticSampleCollectionYea']

In [36]:
def create_date_column_3(df, year_col, original_cols):
    parts = year_col.split('Yea')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Mon' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [37]:
# Convert 'NA' strings to np.nan
for col in col_list:
    if col in df.columns:
        df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in col_list:
    if 'Yea' in col:
        create_date_column_3(df, col, original_cols)

print(df.info())
df.head()

/tmp/ipykernel_28550/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-02' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_28550/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-02' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_28550/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-08-12' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 691 entries, MaskID to MechanisticSampleCollectionDate
dtypes: float64(318), int64(1), object(372)
memory usage: 1.3+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,TimeOfCollectionHH,TimeOfCollectionMM,DateResultsReportedByLabUnknow,TimeResultsProcessedByLabHH,TimeResultsProcessedByLabMM,...,BirthDate,DateOfBirthDate,HbA1crecordDateDate,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,May 1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sep 1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Aug 1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Jan 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sep 1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
len(col_list)

# Drop the columns from the DataFrame
for i in col_list:
    if i in df.columns:
        df.drop(columns=i, inplace=True)
    else:
        continue
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 679 entries, MaskID to MechanisticSampleCollectionDate
dtypes: float64(313), int64(1), object(365)
memory usage: 1.3+ GB


## 1.5 D, M, Y

In [39]:

def create_date_from_components(row):
    # Extracting year, month, and day from the row
    year = row['DateSignedInformedConsentFormY']
    month = row['DateSignedInformedConsentFormM']
    day = row['DateSignedInformedConsentFormD']

    # Check for NaN values and convert floats to integers
    if pd.isna(year) or pd.isna(month) or pd.isna(day):
        return pd.NaT
    try:
        year = int(year)
        day = int(day)
        # Constructing the date string
        date_str = f"{year}-{month}-{day:02d}"
        # Converting to datetime
        return pd.to_datetime(date_str, format='%Y-%b-%d').date()
    except (ValueError, TypeError):
        return pd.NaT

# Apply the function to each row to create the new date column
df['DateSignedInformedConsentFormDate'] = df.apply(create_date_from_components, axis=1)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 680 entries, MaskID to DateSignedInformedConsentFormDate
dtypes: float64(313), int64(1), object(366)
memory usage: 1.3+ GB


In [41]:
# Drop the columns from the DataFrame
df.drop(columns=['DateSignedInformedConsentFormM', 'DateSignedInformedConsentFormD', 'DateSignedInformedConsentFormY'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 677 entries, MaskID to DateSignedInformedConsentFormDate
dtypes: float64(311), int64(1), object(365)
memory usage: 1.3+ GB


## 1.6 Columns without indication of day, month, year

In [42]:
date_dict_1 ={ 'DateSignedNEWInformedConsentFoDate': ['DateSignedNEWInformedConsentFo', 'DateSignedNEWInformedConsentFo2', 'DateSignedNEWInformedConsentFo3'],
             # 'InterferonGammaReleaseAssayIGRDate': ['InterferonGammaReleaseAssayIGR', 'InterferonGammaReleaseAssayIGR3', 'InterferonGammaReleaseAssayIGR2']
             }

In [43]:
# Function to convert month from short name to number
def month_to_num_2(short_month):
    return {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }[short_month]

# Iterating over the dictionary to create new columns in filtered_df
for date_col, cols in date_dict_1.items():
    day_col, month_col, year_col = cols

    # Create a mask for rows where none of the day, month, or year values are missing
    mask = df[[day_col, month_col, year_col]].notna().all(axis=1)

    # Apply the mask to filter out rows with missing values
    valid_rows = df[mask]

    # Iterate over each row to handle individual errors
    for index, row in valid_rows.iterrows():
        try:
            # Combine the date components for the current row
            combined_date = str(int(row[day_col])) + '/' + \
                            str(month_to_num_2(row[month_col])) + '/' + \
                            str(int(row[year_col]))
            
            # Convert to datetime
            df.at[index, date_col] = pd.to_datetime(combined_date, format='%d/%m/%Y')
        except ValueError as e:
            # Print the problematic date and its index
            print(f"Error at index {index}: {combined_date}")
            print(f"Error message: {e}")

# Display the new DataFrame
print(df.info())
df.head()

Error at index 12142: 31/4/2012
Error message: day is out of range for month, at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Error at index 128495: 31/2/2012
Error message: day is out of range for month, at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Error at index 150436: 31/4/2014
Error message: day is out of range for month, at position 0. You might want to try:
    - 

,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,TimeOfCollectionHH,TimeOfCollectionMM,DateResultsReportedByLabUnknow,TimeResultsProcessedByLabHH,TimeResultsProcessedByLabMM,...,HbA1crecordDateDate,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate,DateSignedInformedConsentFormDate,DateSignedNEWInformedConsentFoDate
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2015-06-29
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2015-08-11
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2016-11-21
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2016-10-21


In [44]:
columns_to_drop = [column for sublist in date_dict_1.values() for column in sublist]
print(len(columns_to_drop))

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 675 entries, MaskID to DateSignedNEWInformedConsentFoDate
dtypes: datetime64[ns](1), float64(309), int64(1), object(364)
memory usage: 1.3+ GB


## 1.7 time

In [45]:
time_dict = {'OGTTTestStartTime': ['OGTTTestStartTimeHrs', 'OGTTTestStartTimeMins'],
             'MMTTTestStartTime': ['MMTTTestStartTimeHrs', 'MMTTTestStartTimeMins'],
             'TimeOfCollection': ['TimeOfCollectionHH', 'TimeOfCollectionMM'],
             'TimeResultsProcessedByLab': ['TimeResultsProcessedByLabHH', 'TimeResultsProcessedByLabMM']
            }

In [46]:
def create_time_columns(df, time_dict):
    for new_col, time_cols in time_dict.items():
        hour_col, minute_col = time_cols

        if hour_col in df.columns and minute_col in df.columns:
            # Creating a time column with handling missing values and converting to integers
            df[new_col] = df.apply(lambda row: 
                                   f"{int(row[hour_col]):02d}:{int(row[minute_col]):02d}"
                                   if pd.notna(row[hour_col]) and pd.notna(row[minute_col]) and 
                                      row[hour_col] != '.' and row[minute_col] != '.'
                                   else np.nan, axis=1)
            # Converting string to datetime.time object
            df[new_col] = pd.to_datetime(df[new_col], format='%H:%M', errors='coerce').dt.time
        else:
            print(f"Columns for '{new_col}' not found. Hour column: {hour_col}, Minute column: {minute_col}")

    return df

df = create_time_columns(df, time_dict)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 679 entries, MaskID to TimeResultsProcessedByLab
dtypes: datetime64[ns](1), float64(309), int64(1), object(368)
memory usage: 1.3+ GB
None


,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,TimeOfCollectionHH,TimeOfCollectionMM,DateResultsReportedByLabUnknow,TimeResultsProcessedByLabHH,TimeResultsProcessedByLabMM,...,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate,DateSignedInformedConsentFormDate,DateSignedNEWInformedConsentFoDate,OGTTTestStartTime,MMTTTestStartTime,TimeOfCollection,TimeResultsProcessedByLab
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,2015-06-29,NaT,NaT,NaT,NaT
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,2015-08-11,NaT,NaT,NaT,NaT
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,2016-11-21,NaT,NaT,NaT,NaT
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,2016-10-21,NaT,NaT,NaT,NaT


In [47]:
columns_to_drop = [column for sublist in time_dict.values() for column in sublist]
print(len(columns_to_drop))

for i in columns_to_drop:
    if i in df.columns:
        df.drop(columns=i, inplace=True)
df.info()

8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 671 entries, MaskID to TimeResultsProcessedByLab
dtypes: datetime64[ns](1), float64(301), int64(1), object(368)
memory usage: 1.3+ GB


In [48]:
#df.to_csv('df_column_harmonization.csv', index = False)

In [49]:
df.to_csv('df_column_harmonization.csv',index=False)

In [205]:
df=pd.read_csv('df_column_harmonization.csv')

/tmp/ipykernel_10651/821926730.py:1: DtypeWarning: Columns (3,28,30,32,35,37,40,42,45,47,50,52,93,120,179,180,181,182,183,185,187,188,189,190,194,196,198,200,201,204,205,207,208,209,210,213,214,223,224,225,226,227,228,229,230,231,232,233,242,243,244,245,246,247,256,257,350,351,355,356,357,361,362,363) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('df_column_harmonization.csv')


In [50]:
df

,MaskID,Visit_Dt,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,ProtocolVersion,DateResultsReportedByLabUnknow,Rbcresults,Rbcresults_Units,RBCResultsNormal,RBCResultAbormalClinicallySign,...,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate,DateSignedInformedConsentFormDate,DateSignedNEWInformedConsentFoDate,OGTTTestStartTime,MMTTTestStartTime,TimeOfCollection,TimeResultsProcessedByLab
0,200000,06/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,200003,06/29/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015-06-29,NaN,NaN,NaN,NaN
2,200004,08/11/2015,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015-08-11,NaN,NaN,NaN,NaN
3,200007,11/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016-11-21,NaN,NaN,NaN,NaN
4,200010,10/21/2016,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016-10-21,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259163,879417,12/30/2015,NaN,NaN,New,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,08:50:00,NaT,NaT
259164,879417,07/01/2016,NaN,NaN,New,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,08:20:00,NaT,NaT
259165,879417,12/30/2016,NaN,NaN,New,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,09:20:00,NaT,NaT
259166,879417,06/23/2017,NaN,NaN,New,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,08:55:00,NaT,NaT


# Missing values

In [51]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 666 entries, MaskID to TimeResultsProcessedByLab
dtypes: datetime64[ns](1), float64(296), int64(1), object(368)
memory usage: 1.3+ GB


In [53]:
#identify categorical columns and numeric columns first 

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64']).columns

print("Categorical columns:", categorical_columns)
print('number of categorical columns:',len(categorical_columns))
print("Numerical columns:", numerical_columns) 
print('number of numerical columns:',len(numerical_columns))
print('number of date columns:',len(date_columns))

Categorical columns: Index(['Visit_Dt', '_15Aug2011ConsentForResidualSa', 'WhichProtocolVersion',
       'ProtocolVersion', 'DateResultsReportedByLabUnknow', 'Rbcresults_Units',
       'RBCResultsNormal', 'RBCResultAbormalClinicallySign',
       'HemoglobinResults_Units', 'HemoglobinResultsNormal',
       ...
       'DateRecordedDate3', 'ParticipantDateDispensedDate1_1',
       'ParticipantDateDispensedDate2_1', 'ParticipantDateDispensedDate3_1',
       'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate',
       'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection',
       'TimeResultsProcessedByLab'],
      dtype='object', length=368)
number of categorical columns: 368
Numerical columns: Index(['MaskID', 'Rbcresults', 'HemoglobinResults', 'HematocritResults',
       'MCVresults', 'PlateletCountResultsNormal', 'MCHresults', 'MCHCresults',
       'WBCresults', 'NeutrophilsResults',
       ...
       'BIC', 'ALT', 'ALP', 'ALB', 'UREA', 'AST', 'CPEP', 'INST', 'He

In [54]:
#1st handle wrong datatype
#for any categorical variable, if it contains more than 20 unique values, then we can use common sense to judge 
# if it is better to convert it to 'int' datatype.
for i in categorical_columns:
    if len(df[i].unique())>20:
        print(i)
        print(df[i].unique())

Visit_Dt
['06/29/2007' '06/29/2015' '08/11/2015' ... '10/24/2010' '08/28/2011'
 '04/26/2009']
InitialScreeningLocationOtherS
[nan 'JDRF ONE NATION' 'Screening event' 'Bay Area Diabetes Summit'
 'Offsite Screening Event' 'Offsite' 'Family Diabetes Event'
 'T1D Conference' 'FL AQUARIUM' 'offsite' 'DR. xxxxxx OFFICE'
 'SUMMER BLAST' 'M Health Maple Grove' 'Conference' 'JDRF'
 '2017 CWD Falls Church, VA' 'JDRF Summit' 'Primary Health Care Clinic'
 'PEA-ATLANTA' 'JDRF Type 1 Nation Summit' 'Primary health care clinic'
 'Festival' 'CWD' 'Home' 'JDRF One Nation Event' 'TEDDY site'
 'local diabetes event' 'Home Visit' 'HOME' 'Teddy Malmo Gertie'
 'SEED Office' 'primary health care clinic' 'JDRF Summit mtg' 'Zoo Event'
 'Chart Center' 'Travel Clinic - Pikeville, KY' 'JDRF summit'
 'JDRF Picnic' 'JDRF Conference' 'Diabetic Clinic' 'LIONS CAMP'
 'DESCHUTES COUNTY PUBLIC LIBRARY' 'TEDDY clinic Kristianstad'
 'TypeOneNation Summit, Central Coast (California)' 'JDRF Event'
 'at the Meeting in Carlsb

Date_of_Registration
[nan '03/31/2016' '03/17/2016' '04/14/2016' '05/02/2016' '12/06/2016'
 '04/11/2016' '12/15/2015' '08/04/2016' '08/12/2016' '04/20/2016'
 '05/23/2016' '06/29/2016' '10/14/2016' '05/06/2016' '07/15/2016'
 '12/23/2016' '07/14/2016' '08/01/2016' '01/11/2017' '07/06/2016'
 '08/17/2016' '11/11/2016' '07/21/2016' '04/13/2016' '07/22/2016'
 '06/24/2016' '12/21/2016' '05/11/2016' '10/24/2016' '06/14/2016'
 '08/03/2016' '10/18/2016' '01/05/2016' '07/11/2016' '05/31/2016'
 '12/19/2016' '11/04/2016' '08/19/2016' '06/21/2016' '12/12/2016'
 '05/13/2016' '09/28/2016' '06/06/2016' '08/09/2016' '05/12/2016'
 '10/05/2016' '06/17/2016' '09/13/2016' '05/10/2016' '09/19/2016'
 '04/25/2016' '12/20/2016' '03/25/2016' '05/17/2016' '04/27/2016'
 '05/04/2016' '04/21/2016' '12/16/2016' '07/05/2016' '12/22/2016'
 '09/06/2016' '09/12/2016' '04/07/2016' '04/06/2016' '09/14/2016'
 '01/31/2017' '01/03/2017' '12/28/2015' '12/13/2016' '05/25/2016'
 '06/09/2016' '06/05/2016' '10/17/2016' '06/02/2016

BirthDate
['May 1987' 'Sep 1994' 'Aug 1970' 'Jan 2009' 'Sep 1982' 'Sep 2000'
 'Oct 1997' 'Jul 1989' 'Jul 1961' 'Nov 1990' 'May 1963' 'Jun 1971'
 'Jul 2012' 'Jul 1981' 'Jul 1978' 'Sep 1973' 'Jul 1977' 'Feb 2005'
 'Dec 1969' 'May 1981' 'Mar 2000' 'Aug 1985' 'Dec 2010' 'Aug 1999'
 'Mar 1968' 'Jun 2007' 'Jun 1999' 'Aug 2004' 'Jun 2008' 'Jan 1994'
 'Apr 1966' 'Sep 1967' 'Jan 2000' 'Oct 1975' 'Nov 1995' 'Nov 1988'
 'May 2010' 'Oct 1993' 'Oct 2001' 'Mar 2011' 'Jan 1986' 'Nov 2000'
 'May 1997' nan 'Sep 2007' 'May 1985' 'Feb 1964' 'Jul 2007' 'Dec 2006'
 'Feb 1992' 'Jan 2002' 'Sep 2006' 'Jul 1972' 'Jan 2004' 'Oct 2002'
 'Dec 1976' 'Jan 2015' 'Aug 2005' 'May 2008' 'May 1989' 'Feb 2003'
 'Jul 2006' 'Dec 2015' 'Jan 1982' 'Apr 1989' 'Oct 2013' 'Mar 2004'
 'Nov 1976' 'Feb 1978' 'Oct 1971' 'Mar 2001' 'Sep 1974' 'Aug 1996'
 'Oct 1989' 'Dec 1990' 'Jan 1997' 'Oct 2009' 'Sep 1985' 'Dec 1970'
 'Aug 1982' 'Dec 1995' 'Sep 1970' 'Mar 1986' 'May 1975' 'Nov 2006'
 'Apr 2000' 'Aug 1969' 'Oct 1992' 'Apr 1998' 'Ja

In [55]:
for i in numerical_columns:
    if len(df[i].unique())<10:
        print(i)
        print(df[i].unique())

DegreeOfRelative_1st
[ 0.  1. nan]
DegreeOfRelative_2ndor3rd
[ 1.  0. nan]
Race_AmericanIndianorAlaskaNat
[ 0. nan  1.]
Race_Asian
[ 0.  1. nan]
Race_BlackorAfricanAmerican
[ 0. nan  1.]
Race_NativeHawaiianorOtherPaci
[ 0. nan  1.]
Race_Refused
[ 0. nan  1.]
Race_Unknownornotreported
[ 1.  0. nan]
Race_White
[ 0.  1. nan]
FirstHearAboutTrialN_ExistingTri
[ 0.  1. nan]
FirstHearAboutTrialN_FamilyFrien
[ 0.  1. nan]
FirstHearAboutTrialN_Brochure
[ 0.  1. nan]
FirstHearAboutTrialN_Healthcarep
[ 1.  0. nan]
FirstHearAboutTrialN_Tollfreepho
[ 0. nan  1.]
FirstHearAboutTrialN_NewspaperMa
[ 0. nan  1.]
FirstHearAboutTrialN_Poster
[ 0. nan  1.]
FirstHearAboutTrialN_Camp
[ 0.  1. nan]
FirstHearAboutTrialN_MeetingPres
[ 0.  1. nan]
FirstHearAboutTrialN_RadioTV
[ 0. nan  1.]
FirstHearAboutTrialN_Internet
[ 0.  1. nan]
FirstHearAboutTrialN_Other
[ 0.  1. nan]
IsHeightMeasured
[nan  1.]
IsWeightMeasured
[nan  1.]
IfSpecimensNotCollec_Other
[nan  1.]
OGTTTimeToConsumeGlucola
[nan  2.  4.  1.  3.  5.

In [56]:
#these columns only have  value of 0 in it， so we can just drop them
element_to_remove =['EBVVCAIGM','EBVVCAIGG','EBNA','CMVIGM','HEPC','CMVIGG','HIV', 'HEPBC','HEPB','DR3','DR4','HLA','HLAa','HLAb']

for i in element_to_remove:
    df=df.drop(i,axis=1)


In [57]:
#columns should be coverted to categorical variables 
to_convert_cat=['DateResultsReportedByLabUnknow',
            'DegreeOfRelative_1st',
            'DegreeOfRelative_2ndor3rd',
            'Race_AmericanIndianorAlaskaNat',
            'Race_Asian',
            'Race_BlackorAfricanAmerican',
            'Race_NativeHawaiianorOtherPaci',
            'Race_Refused',
            'Race_Unknownornotreported',
            'Race_White',
            'FirstHearAboutTrialN_ExistingTri',
            'FirstHearAboutTrialN_FamilyFrien',
            'FirstHearAboutTrialN_Brochure',
            'FirstHearAboutTrialN_Healthcarep',
            'FirstHearAboutTrialN_Tollfreepho',
            'FirstHearAboutTrialN_NewspaperMa',
            'FirstHearAboutTrialN_Poster',
            'FirstHearAboutTrialN_Camp',
            'FirstHearAboutTrialN_MeetingPres',
            'FirstHearAboutTrialN_RadioTV',
            'FirstHearAboutTrialN_Internet',
            'FirstHearAboutTrialN_Other',
            
            'Rbcnotdone',
            'HemoglobinNotDone','HematocritNotDone','MCVNotDone',
            'WBCNotDone','bloodpressureDiastolicNotDone','WeightNotDone',
            'HeightNotDone']


In [58]:
#convert some numerical variables to categorical variables 
for i in to_convert_cat:
    df[i] = df[i].astype(object)


In [74]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64[ns]']).columns
#statistic the columns with <50% missing value; >=50% & <=99% missing value; >99% missing value
missing_60=[]
drop_95=[]
between_50_99=[]
for i in numerical_columns:
    count_null=df[i].isnull().sum()
    print(i)
    all1=len(df[i])
    null_per=count_null/all1
    print(null_per*100,'%')
    if null_per*100<50:
        missing_60.append(i)
    if null_per*100>=50 and null_per*100<=99:
        between_50_99.append(i)
    if null_per*100>99:
        drop_95.append(i)
     #   df = df.drop(i, axis=1)


MaskID
0.0 %
Rbcresults
99.22984319051736 %
HemoglobinResults
99.22984319051736 %
HematocritResults
99.2310007408322 %
MCVresults
99.22984319051736 %
PlateletCountResultsNormal
99.23215829114706 %
MCHresults
99.23061489072725 %
MCHCresults
99.23833189282627 %
WBCresults
99.2310007408322 %
NeutrophilsResults
99.60566119273984 %
NeutrophilsResultsOR
99.61646499567848 %
LymphocytesResults
99.59022718854179 %
LymphocytesResultsOR
99.60488949252995 %
MonocytesResults
99.5925422891715 %
MonocytesResultsOR
99.60450364242499 %
EosinophilsResults
99.59369983948636 %
EosinophilsResultsOR
99.60450364242499 %
BasophilsResults
99.5948573898012 %
BasophilsResultsOR
99.60720459315965 %
Age
9.927537350290159 %
clinicalcenternumber
65.77663909124584 %
ScreeningSiteNumber
30.9991974317817 %
NumOfRelativesWithT1D
18.32054883318928 %
AgeOfRelativeWithT1D1_1
16.348083096678604 %
AgeOfOnsetInRelativeWithT1D1_1
13.854333868378813 %
AgeRelativeWithT1DStartedIn1_1
16.417536115569824 %
AgeOfRelativeWithT1D2_1
8

In [78]:
df=df.drop(columns=drop_95)

In [68]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259168 entries, 0 to 259167
Columns: 652 entries, MaskID to TimeResultsProcessedByLab
dtypes: datetime64[ns](1), float64(261), int64(1), object(389)
memory usage: 1.3+ GB


In [75]:
len(missing_60)

73

In [76]:
len(drop_95)

175

In [77]:
len(between_50_99)

14

In [80]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns


In [81]:
# get date columns out of the categorical columns. Use the remaining categorical columns for encoding
date_col = [col for col in categorical_columns]

def separate_columns_by_date(column_names):
    
    date_columns = []
    non_date_columns = []

    # Regular expression pattern for common date-related keywords
    date_pattern = re.compile(r'date|time', re.IGNORECASE)

    for column_name in column_names:
        if date_pattern.search(column_name):
            date_columns.append(column_name)
        else:
            non_date_columns.append(column_name)

    return date_columns, non_date_columns


date_columns, non_date_columns = separate_columns_by_date(date_col)
#these two columns are also date columns
date_columns.extend(['Visit_Dt'])

element_to_remove =['Visit_Dt']
for i in element_to_remove:
    if i in non_date_columns:
        non_date_columns.remove(i)


print("Columns that likely represent dates:")
print(date_columns)

print("\nColumns that do not appear to represent dates:")
print(non_date_columns)


Columns that likely represent dates:
['DateResultsReportedByLabUnknow', 'Date_of_Registration', 'RememberDateOfMostRecentHbA1c', 'DateOfRegistration', 'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM', 'TimeDoseGivenHr', 'ParticipantDateDispensedYea4_1', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate', 'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate', 'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate', 'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate', 'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3', 'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1', 'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate', 'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection', 'TimeResultsProcessedByLab', 'Visit_D

In [82]:
len(date_columns)+len(non_date_columns)

389

In [108]:
len(non_date_columns)

351

In [83]:
numerical_columns = df.select_dtypes(include=['int', 'float']).columns

In [84]:
len(numerical_columns)

87

In [85]:
#fill in zero for the null values in Date/Time columns
df[date_columns]=df[date_columns].fillna(0)

In [86]:
#encode all the cateogrical variables except date and time

embed=[]
for i in non_date_columns:
    print(i)
    df[i]=df[i].astype('str')
    column_name=i
    value_counts = df[column_name].value_counts()
    #print(value_counts)
    my_dict={}
    key=df[column_name].unique()
    print('unique values:',key)
    null_count = df[column_name].isnull().sum()
    #print(f"Number of null values in column '{column_name}': {null_count}")
    le = LabelEncoder()
    df[column_name] = le.fit_transform(df[column_name])
    value=df[column_name].unique()
    # Fill the dictionary with keys and values using a loop
    for k, v in zip(key, value):
        my_dict[k] = v

    dict1={'Column':column_name,'Details':my_dict}
    value_counts = df[column_name].value_counts()
   # print(value_counts)
    null_count = df[column_name].isnull().sum()
    print(f"Number of null values after preprocessing in column '{column_name}': {null_count}")
    
    embed.append(dict1)

_15Aug2011ConsentForResidualSa
unique values: ['nan' 'Yes' 'No' 'Unknown']
Number of null values after preprocessing in column '_15Aug2011ConsentForResidualSa': 0
WhichProtocolVersion
unique values: ['nan' 'October 10, 2017' '22February2019']
Number of null values after preprocessing in column 'WhichProtocolVersion': 0
ProtocolVersion
unique values: ['nan' '3.0' '2.0' '4.0' 'New' 'Old']
Number of null values after preprocessing in column 'ProtocolVersion': 0
Rbcresults_Units
unique values: ['NaN' 'x10^12 cells/L']
Number of null values after preprocessing in column 'Rbcresults_Units': 0
RBCResultsNormal
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'RBCResultsNormal': 0
RBCResultAbormalClinicallySign
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'RBCResultAbormalClinicallySign': 0
HemoglobinResults_Units
unique values: ['NaN' 'mg/dL']
Number of null values after preprocessing in column 'HemoglobinResults_Un

Number of null values after preprocessing in column 'DegreeOfRelative_2ndor3rd': 0
HaveDiabetes
unique values: ['No' 'Yes' 'nan']
Number of null values after preprocessing in column 'HaveDiabetes': 0
ParticipantLocation
unique values: ['nan' 'International' 'North American']
Number of null values after preprocessing in column 'ParticipantLocation': 0
Sex
unique values: ['Male' 'Female' 'nan']
Number of null values after preprocessing in column 'Sex': 0
Ethnicity
unique values: ['Hispanic or Latino' 'Not Hispanic or Latino' 'Unknown' 'nan'
 'Not Hispanic, Latino, or Spanish Origin'
 'Hispanic, Latino, or Spanish Origin']
Number of null values after preprocessing in column 'Ethnicity': 0
Race_AmericanIndianorAlaskaNat
unique values: ['0.0' 'nan' '1.0']
Number of null values after preprocessing in column 'Race_AmericanIndianorAlaskaNat': 0
Race_Asian
unique values: ['0.0' '1.0' 'nan']
Number of null values after preprocessing in column 'Race_Asian': 0
Race_BlackorAfricanAmerican
unique va

unique values: ['Yes' 'nan' 'No' 'Unknown']
Number of null values after preprocessing in column 'BloodSamplesFutureTestingPermi': 0
BloodTestForFindOut
unique values: ['nan'
 'I have autoantibodies which means I am at risk of getting Type 1 diabetes'
 'I have Type 1 diabetes' 'I don?t know' 'I have Type 2 diabetes']
Number of null values after preprocessing in column 'BloodTestForFindOut': 0
CommunicationWhenNegAutoantibo
unique values: ['nan' 'Letter' 'Phone Call'
 'We will only notify you if you are positive for autoantibodies'
 'I don?t know']
Number of null values after preprocessing in column 'CommunicationWhenNegAutoantibo': 0
InformedTestResultWithin
unique values: ['nan' '4-6 weeks' '1 week' '3 months' '1 year']
Number of null values after preprocessing in column 'InformedTestResultWithin': 0
PermissionToControlIfTheyTestA
unique values: ['nan' 'No' 'Unknown' 'Yes']
Number of null values after preprocessing in column 'PermissionToControlIfTheyTestA': 0
participanthaveautoantibo

Number of null values after preprocessing in column 'VitiligoEver': 0
VitiligoWithinLastYear
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'VitiligoWithinLastYear': 0
ThyroidDiseaseEver
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'ThyroidDiseaseEver': 0
ThyroidDiseaseWithinLastYear
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'ThyroidDiseaseWithinLastYear': 0
PsoriasisEver
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'PsoriasisEver': 0
PsoriasisWithinLastYear
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'PsoriasisWithinLastYear': 0
AlopeciaEver
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'AlopeciaEver': 0
AlopeciaWithinLastYear
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'AlopeciaWithinLastYear': 0
RheumatologicEver
unique values: ['nan' 'No']

Number of null values after preprocessing in column 'AddisonsDiseaseWithinLast6Months': 0
VitiligoWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'VitiligoWithinLast6Months': 0
ThyroidDiseaseWithinLast6Months
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'ThyroidDiseaseWithinLast6Months': 0
PsoriasisWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'PsoriasisWithinLast6Months': 0
AlopeciaWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'AlopeciaWithinLast6Months': 0
RheumatologicWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'RheumatologicWithinLast6Months': 0
AllergiesWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'AllergiesWithinLast6Months': 0
CancerWithinLast6Months
un

Number of null values after preprocessing in column 'NeckThyroidFindings': 0
HeartFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'HeartFindings': 0
MusculoskeletalFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'MusculoskeletalFindings': 0
GenitaliaFindings
unique values: ['nan' 'Normal' 'Not Assessed']
Number of null values after preprocessing in column 'GenitaliaFindings': 0
AbdomenFindings
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'AbdomenFindings': 0
LymphaticsFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'LymphaticsFindings': 0
SkinFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'SkinFindings': 0
NeurologicFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after prepro

Number of null values after preprocessing in column 'Rbcnotdone': 0
HemoglobinNotDone
unique values: ['nan' '1']
Number of null values after preprocessing in column 'HemoglobinNotDone': 0
HematocritNotDone
unique values: ['nan' '1']
Number of null values after preprocessing in column 'HematocritNotDone': 0
MCVNotDone
unique values: ['nan' '1']
Number of null values after preprocessing in column 'MCVNotDone': 0
WBCNotDone
unique values: ['nan' '1']
Number of null values after preprocessing in column 'WBCNotDone': 0
Doestheparticipanthavereproduc
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'Doestheparticipanthavereproduc': 0
Wasaurinepregnancytestcomplete
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'Wasaurinepregnancytestcomplete': 0
Wasthetestresultpositive
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'Wasthetestresultpositive': 0
bloodpressureDiastolicNotDone
unique va

In [88]:
#save it to create data dictionary
#with open('encoding.pickle', 'wb') as file:
#    pickle.dump(embed, file)

In [89]:
# Loop through the specified columns and add a missing indicator column for each
# 1 means misssing , 0 means not missing
for col in between_50_99:
    df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)

/tmp/ipykernel_28550/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_28550/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_28550/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

In [93]:
#fill NaN with 0 for the columns with missing values between 50% and 99%
df[between_50_99]=df[between_50_99].fillna(0)

In [96]:
df.shape

(259168, 491)

In [ ]:
# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust 'n_neighbors' as needed
# Impute missing values using the KNN imputer for each numeric column with less than 50% missing value
j=0
for column in missing_60[1:]:
    print(column)
    print(j)
    j+=1
    df[column] = imputer.fit_transform(df[column].values.reshape(-1, 1))

GAD65H_y
0
MIAA
1
IA-2H
2
ZNT8
3
ICA512
4
ICA
5
GAD65
6
GLU60
7
PEP120
8
PEP60
9
INST-10
10
PEP-10
11
INST60
12
INST0
13
GLU30
14
HbA1c
15
GLU-10
16
GLU0
17
PEP30
18
INST30
19
INST120
20
GLU90
21
PEP90
22
GLU120
23
INST90
24
PEP0
25
INS0
26
INS120
27
INS-10
28
INS60
29
INS30
30
INS90
31
GLU15
32
PEP15
33
PEP240
34
GLU240
35
PEP150
36
GLU180
37
PEP180
38
GLU150
39
PEP210
40
EBVVL
41
CMVVL
42
GLU210
43
CD8P
44
CD8A
45
CD4P
46
CD4CD8
47
CD4A
48
TPRO
49
TBIL
50
SOD
51
POT
52
GLU
53
GLOIgG
54
DBIL
55
CREA
56
CHL
57
BIC
58
ALT
59
ALP
60
ALB
61
UREA
62


In [ ]:
df.to_csv('AI_Ready.csv',index=False)

In [227]:
d=pd.read_csv('AI_Ready.csv')

/tmp/ipykernel_28550/2546570015.py:1: DtypeWarning: Columns (85,226,241,278,289,309,436,437,438,439,440,441,442,444,445,446,447,448,449,450,451,455,456,457,458,459) have mixed types. Specify dtype option on import or set low_memory=False.
  d=pd.read_csv('AI_Ready.csv')


In [229]:
# Check if there are any null values in the DataFrame
if d.isnull().any().any():
    print("There are null values in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

There are no null values in the DataFrame.


In [222]:
d[columns_with_null]=df[columns_with_null].fillna(0)

In [201]:
dfc=pd.merge(df1,old,on='Variable Name',how='left')
dfc

,Variable Name,Details,Description,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation,Variable Manipulation
0,_15Aug2011ConsentForResidualSa,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",Did participant consent to continued storage o...,object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation,Yes
1,WhichProtocolVersion,"{'nan': 2, 'October 10, 2017': 1, '22February2...",Which version of the Natural History protocol ...,object,NaN,"{'nan': 4, '15August2011': 0, '22July2009': 2,...",KNN Imputation,Yes
2,ProtocolVersion,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",Protocol Version,object,NaN,"{'nan': 7, '3.0': 2, '4.0': 3, '2.0': 1, 'Octo...",KNN Imputation,Yes
3,Rbcresults_Units,"{'NaN': 0, 'x10^12 cells/L': 1}",Unit of RBC results,object,NaN,"{'nan': 0, 'x10^12 cells/L': 1}",KNN Imputation,Yes
4,RBCResultsNormal,"{'nan': 2, 'Yes': 1, 'No': 0}",RBC results normal,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
...,...,...,...,...,...,...,...,...
347,STIstheparticipantcurrentlytak,"{'nan': 2, 'Yes': 1, 'No': 0}",ST Is the participant currently taking study drug,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
348,STWastheparticipantcontactedat,"{'nan': 2, 'Yes': 1, 'No': 0}",ST Was the participant contacted at a minimum ...,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
349,HbA1c_Units,"{'Hb%': 0, 'NaN': 1}",Unit of HbA1c,object,NaN,"{'nan': 1, 'Hb%': 0}",KNN Imputation,Yes
350,Height_cm_units,{'cm': 0},unit of height,object,NaN,{'cm': 0},KNN Imputation,Yes


In [202]:
dfc.drop('Corresponding Code List',axis=1,inplace=True)
dfc

,Variable Name,Details,Description,Variable Type,Measurement Units as applicable,Missingness Representation,Variable Manipulation
0,_15Aug2011ConsentForResidualSa,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",Did participant consent to continued storage o...,object,NaN,KNN Imputation,Yes
1,WhichProtocolVersion,"{'nan': 2, 'October 10, 2017': 1, '22February2...",Which version of the Natural History protocol ...,object,NaN,KNN Imputation,Yes
2,ProtocolVersion,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",Protocol Version,object,NaN,KNN Imputation,Yes
3,Rbcresults_Units,"{'NaN': 0, 'x10^12 cells/L': 1}",Unit of RBC results,object,NaN,KNN Imputation,Yes
4,RBCResultsNormal,"{'nan': 2, 'Yes': 1, 'No': 0}",RBC results normal,object,NaN,KNN Imputation,Yes
...,...,...,...,...,...,...,...
347,STIstheparticipantcurrentlytak,"{'nan': 2, 'Yes': 1, 'No': 0}",ST Is the participant currently taking study drug,object,NaN,KNN Imputation,Yes
348,STWastheparticipantcontactedat,"{'nan': 2, 'Yes': 1, 'No': 0}",ST Was the participant contacted at a minimum ...,object,NaN,KNN Imputation,Yes
349,HbA1c_Units,"{'Hb%': 0, 'NaN': 1}",Unit of HbA1c,object,NaN,KNN Imputation,Yes
350,Height_cm_units,{'cm': 0},unit of height,object,NaN,KNN Imputation,Yes


In [203]:
# Step 1: Extract the second column ('B')
second_column = dfc.pop('Details')

# Step 2: Remove the second column from the DataFrame
# Now, df does not have the 'B' column

# Step 3: Insert the second_column Series at the fifth position
dfc.insert(4, 'Details', second_column)

In [204]:
dfc

,Variable Name,Description,Variable Type,Measurement Units as applicable,Details,Missingness Representation,Variable Manipulation
0,_15Aug2011ConsentForResidualSa,Did participant consent to continued storage o...,object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation,Yes
1,WhichProtocolVersion,Which version of the Natural History protocol ...,object,NaN,"{'nan': 2, 'October 10, 2017': 1, '22February2...",KNN Imputation,Yes
2,ProtocolVersion,Protocol Version,object,NaN,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",KNN Imputation,Yes
3,Rbcresults_Units,Unit of RBC results,object,NaN,"{'NaN': 0, 'x10^12 cells/L': 1}",KNN Imputation,Yes
4,RBCResultsNormal,RBC results normal,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
...,...,...,...,...,...,...,...
347,STIstheparticipantcurrentlytak,ST Is the participant currently taking study drug,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
348,STWastheparticipantcontactedat,ST Was the participant contacted at a minimum ...,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
349,HbA1c_Units,Unit of HbA1c,object,NaN,"{'Hb%': 0, 'NaN': 1}",KNN Imputation,Yes
350,Height_cm_units,unit of height,object,NaN,{'cm': 0},KNN Imputation,Yes


In [205]:
dfc1=dfc.rename(columns={'Details': 'Corresponding Code List'}, inplace=True)


dfc1

In [207]:
dfc

,Variable Name,Description,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation,Variable Manipulation
0,_15Aug2011ConsentForResidualSa,Did participant consent to continued storage o...,object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation,Yes
1,WhichProtocolVersion,Which version of the Natural History protocol ...,object,NaN,"{'nan': 2, 'October 10, 2017': 1, '22February2...",KNN Imputation,Yes
2,ProtocolVersion,Protocol Version,object,NaN,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",KNN Imputation,Yes
3,Rbcresults_Units,Unit of RBC results,object,NaN,"{'NaN': 0, 'x10^12 cells/L': 1}",KNN Imputation,Yes
4,RBCResultsNormal,RBC results normal,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
...,...,...,...,...,...,...,...
347,STIstheparticipantcurrentlytak,ST Is the participant currently taking study drug,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
348,STWastheparticipantcontactedat,ST Was the participant contacted at a minimum ...,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
349,HbA1c_Units,Unit of HbA1c,object,NaN,"{'Hb%': 0, 'NaN': 1}",KNN Imputation,Yes
350,Height_cm_units,unit of height,object,NaN,{'cm': 0},KNN Imputation,Yes


In [209]:
dfd = d.drop(index=range(440))


In [213]:
import pandas as pd


# Concatenate (union) the two DataFrames
result_df = pd.concat([dfd, dfc])

print(result_df)


                      Variable Name  \
440   willingToConsiderAtALaterTime   
441  DateParticipantwascontacted1_2   
442     HospitalizedAtDiagnosisTime   
443            Date_of_Registration   
444             Date_of_Study_Start   
..                              ...   
347  STIstheparticipantcurrentlytak   
348  STWastheparticipantcontactedat   
349                     HbA1c_Units   
350                 Height_cm_units   
351                 Weight_kg_units   

                                           Description Variable Type  \
440  Did the participant indicate they may be willi...        object   
441                     Date participant was contacted        object   
442                        HospitalizedAtDiagnosisTime        object   
443                               Date of Registration        object   
444                                Date of Study Start        object   
..                                                 ...           ...   
347  ST Is the participant 

In [215]:
result_df.to_csv('data_dictionary.csv',index=False)


In [208]:
d.head(440)


,Variable Name,Description,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation,Variable Manipulation
0,Visit,Visit,object,NaN,"{'PRN': 70, '4 years': 36, 'Baseline': 63, '6 ...",KNN Imputation,Yes
1,_22Jul2009VisitType,NaN,object,NaN,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation,Yes
2,_15Aug2011VisitType,Visit type,object,NaN,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation,Yes
3,StorageOfResidualsDuringTrialN,Did participant consent to current storage of ...,object,NaN,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation,Yes
4,_15Aug2011ConsentForResidualSa,Did participant consent to continued storage o...,object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation,Yes
...,...,...,...,...,...,...,...
435,Havetherebeenanychangesinconco,Have there been any changes in concomitant med...,object,NaN,"{'nan': 2, 'No': 0, 'Yes': 1}",KNN Imputation,Yes
436,Istheparticipantcurrentlytakin,Is the participant currently taking study drug,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation,Yes
437,HbA1c_Units,Unit of HbA1c,object,NaN,"{'nan': 1, 'Hb%': 0}",KNN Imputation,Yes
438,Height_cm_units,unit of height,object,NaN,{'cm': 0},KNN Imputation,Yes


In [112]:
d1=pd.read_csv('df_encode.csv')

In [178]:
d1

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,Visit,object,NaN,"{'PRN': 70, '4 years': 36, 'Baseline': 63, '6 ...",KNN Imputation
1,_22Jul2009VisitType,object,NaN,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation
2,_15Aug2011VisitType,object,NaN,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation
3,StorageOfResidualsDuringTrialN,object,NaN,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation
4,_15Aug2011ConsentForResidualSa,object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation
...,...,...,...,...,...
434,Havetherebeenanychangesinconco,object,NaN,"{'nan': 2, 'No': 0, 'Yes': 1}",KNN Imputation
435,Istheparticipantcurrentlytakin,object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation
436,HbA1c_Units,object,NaN,"{'nan': 1, 'Hb%': 0}",KNN Imputation
437,Height_cm_units,object,NaN,{'cm': 0},KNN Imputation


In [114]:
embed

[{'Column': '_15Aug2011ConsentForResidualSa',
  'Details': {'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}},
 {'Column': 'WhichProtocolVersion',
  'Details': {'nan': 2, 'October 10, 2017': 1, '22February2019': 0}},
 {'Column': 'ProtocolVersion',
  'Details': {'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New': 3, 'Old': 4}},
 {'Column': 'Rbcresults_Units', 'Details': {'NaN': 0, 'x10^12 cells/L': 1}},
 {'Column': 'RBCResultsNormal', 'Details': {'nan': 2, 'Yes': 1, 'No': 0}},
 {'Column': 'RBCResultAbormalClinicallySign',
  'Details': {'nan': 2, 'No': 0, 'Yes': 1}},
 {'Column': 'HemoglobinResults_Units', 'Details': {'NaN': 0, 'mg/dL': 1}},
 {'Column': 'HemoglobinResultsNormal',
  'Details': {'nan': 2, 'Yes': 1, 'No': 0}},
 {'Column': 'HemoglobinResultAbormalClinica',
  'Details': {'nan': 2, 'No': 0, 'Yes': 1}},
 {'Column': 'HematocritResults_Units', 'Details': {'NaN': 1, '% of blood': 0}},
 {'Column': 'HematocritResultsNormal',
  'Details': {'nan': 2, 'Yes': 1, 'No': 0}},
 {'Column': 'HematocritRe

In [126]:
for item in embed:
    item['Details'] = str(item['Details'])  # or list(item['Details'].items())

# Create DataFrame
df1= pd.DataFrame(embed)

df1.rename(columns={'Column'：'Variable Name'})

SyntaxError: invalid character '：' (U+FF1A) (2060851917.py, line 7)

In [127]:
df1=df1.rename(columns={'Column': 'Variable Name'})


In [128]:
df1

,Variable Name,Details
0,_15Aug2011ConsentForResidualSa,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}"
1,WhichProtocolVersion,"{'nan': 2, 'October 10, 2017': 1, '22February2..."
2,ProtocolVersion,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'..."
3,Rbcresults_Units,"{'NaN': 0, 'x10^12 cells/L': 1}"
4,RBCResultsNormal,"{'nan': 2, 'Yes': 1, 'No': 0}"
...,...,...
346,STIstheparticipantcurrentlytak,"{'nan': 2, 'Yes': 1, 'No': 0}"
347,STWastheparticipantcontactedat,"{'nan': 2, 'Yes': 1, 'No': 0}"
348,HbA1c_Units,"{'Hb%': 0, 'NaN': 1}"
349,Height_cm_units,{'cm': 0}


In [131]:



result_df = pd.merge(df1, d1, on='Variable Name', how='left')


In [135]:
result_df

,Variable Name,Details,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,_15Aug2011ConsentForResidualSa,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",object,NaN,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation
1,WhichProtocolVersion,"{'nan': 2, 'October 10, 2017': 1, '22February2...",object,NaN,"{'nan': 4, '15August2011': 0, '22July2009': 2,...",KNN Imputation
2,ProtocolVersion,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",object,NaN,"{'nan': 7, '3.0': 2, '4.0': 3, '2.0': 1, 'Octo...",KNN Imputation
3,Rbcresults_Units,"{'NaN': 0, 'x10^12 cells/L': 1}",object,NaN,"{'nan': 0, 'x10^12 cells/L': 1}",KNN Imputation
4,RBCResultsNormal,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation
...,...,...,...,...,...,...
346,STIstheparticipantcurrentlytak,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation
347,STWastheparticipantcontactedat,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation
348,HbA1c_Units,"{'Hb%': 0, 'NaN': 1}",object,NaN,"{'nan': 1, 'Hb%': 0}",KNN Imputation
349,Height_cm_units,{'cm': 0},object,NaN,{'cm': 0},KNN Imputation


In [138]:
result=result_df.drop('Corresponding Code List',axis=1)

In [141]:
result['Missingness Representation']='add Null as a category'

In [143]:
result['Variable Type']='object'

In [151]:
result=result.rename(columns={'Details':'Corresponding Code List'})

In [152]:
result

,Variable Name,Corresponding Code List,Variable Type,Measurement Units as applicable,Missingness Representation
0,_15Aug2011ConsentForResidualSa,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",object,NaN,add Null as a category
1,WhichProtocolVersion,"{'nan': 2, 'October 10, 2017': 1, '22February2...",object,NaN,add Null as a category
2,ProtocolVersion,"{'nan': 5, '3.0': 1, '2.0': 0, '4.0': 2, 'New'...",object,NaN,add Null as a category
3,Rbcresults_Units,"{'NaN': 0, 'x10^12 cells/L': 1}",object,NaN,add Null as a category
4,RBCResultsNormal,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,add Null as a category
...,...,...,...,...,...
346,STIstheparticipantcurrentlytak,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,add Null as a category
347,STWastheparticipantcontactedat,"{'nan': 2, 'Yes': 1, 'No': 0}",object,NaN,add Null as a category
348,HbA1c_Units,"{'Hb%': 0, 'NaN': 1}",object,NaN,add Null as a category
349,Height_cm_units,{'cm': 0},object,NaN,add Null as a category


In [176]:
result2=pd.DataFrame({'Variable Name':date_columns})

In [177]:
result2['Corresponding Code List']=None
result2['Variable Type']='datetime64'
result2['Measurement Units as applicable']=None
result2['Missingness Representation']=0


In [175]:
result2

,Variable Name,Corresponding Code List,Variable Type,Measurement Units as applicable,Missingness Representation
0,DateSignedNEWInformedConsentFoDate,None,object,None,0


In [168]:

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64']).columns

print("Categorical columns:", categorical_columns)
print('number of categorical columns:',len(categorical_columns))
print("Numerical columns:", numerical_columns) 
print('number of numerical columns:',len(numerical_columns))
print('number of date columns:',len(date_columns))

Categorical columns: Index(['Visit_Dt', 'DateResultsReportedByLabUnknow', 'Date_of_Registration',
       'RememberDateOfMostRecentHbA1c', 'DateOfRegistration',
       'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM',
       'TimeDoseGivenHr', 'ParticipantDateDispensedYea4_1',
       'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate',
       'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate',
       'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate',
       'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate',
       'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate',
       'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate',
       'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3',
       'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1',
       'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate',
       'DateSignedInformedConsentFormDate', 'OGTTT

In [170]:
date_columns

Index(['DateSignedNEWInformedConsentFoDate'], dtype='object')